# **Maxlik and Poisson**

**Alicja_Smaruj**

"Informatyka i ekonomtria" 

studia stacjonarne

rok 3, semestr 6

In [26]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


#Homework


Assume that $X$ follows Poisson distribution given by 
    
  \begin{equation}
      P(X=x, \lambda_i) = \frac{\lambda^x e^{-\lambda}}{x!},
  \end{equation}

  where $\lambda_i = \theta_0 + \theta_1 \times z_i$, $\theta_0=0.5$, $\theta_1=0.5$, and $z_i \sim \text{Bern}(0.7)$ and number of observations is equal to $n=10,000$.

Tasks:

  1. generate $z_i$,
  2. generate $\lambda_i$ according to $\theta_0 + \theta_1 \times z_i$,
  3. generate $X \sim Poisson(\lambda_i)$,
  4. derive log-likelihood, gradient and hessian (by hand, \LaTeX code for equations are preferred),
  5. obtain MLE of $\boldsymbol{\theta} = (\theta_0, \theta_1)$ using Newton-Raphson method (gradient and hessian should be applied). 



In [27]:
%%R
install.packages("maxLik")

R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/Rtmp4pJt4x/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [28]:
%%R
library(maxLik)

1. Generate $z_i \sim \text{Bern}(0.7)$ and number of observations is equal to $n=10,000$


In [30]:
%%R
z <- rbinom(n = 10000, size = 1, prob = 0.7)

2. Generate $\lambda_i$ according to $\theta_0 + \theta_1 \times z_i$,  $\theta_0=0.5$, $\theta_1=0.5$


In [31]:
%%R
lambda <- 0.5 + 0.5*z

3. Generate $X \sim Poisson(\lambda_i)$

In [32]:
%%R
x <- rpois(10000, lambda)

4. Derive log-likelihood, gradient and hessian (by hand, \LaTeX code for equations are preferred)

###**For parameter λ**

Poisson distribution

\begin{equation}
    P(X=x, \lambda_i) = \frac{\lambda^x e^{-\lambda}}{x!},
\end{equation}

Likelihood function for Poisson distribution is:

\begin{equation}
    L = \prod_i^{n} \frac{\lambda^x e^{-\lambda}}{x_i!},
\end{equation}

The log-likelihood function is:

\begin{equation}
   \ln L = -n\lambda + \sum_i^{n} x_i \ln \lambda - \sum_i^{n} \ln(x_i!),
\end{equation}

Gradient:

\begin{equation}
    \frac{\partial \log L}{\partial \lambda} = -n - \frac{\sum_i^{n} x_i}{\lambda},   
\end{equation}

Hessian:

\begin{equation}
    \frac{\partial^2 \log L}{\partial \lambda^2} = \frac{\sum_i^{n} x_i}{\lambda^2}.
\end{equation}

###**For parameters $\boldsymbol{\theta} = (\theta_0, \theta_1)$**

Poisson distribution

\begin{equation}
    P(X=x, \boldsymbol{\theta}) = \frac{(\theta_0 + \theta_1 \times z)^x e^{-\theta_0 - \theta_1 \times z}}{x!},
\end{equation}

Likelihood function for Poisson distribution is:

\begin{equation}
    L = \prod_i^{n} \frac{(\theta_0 + \theta_1 \times z)^x e^{-\theta_0 - \theta_1 \times z}}{x_i!},
\end{equation}

The log-likelihood function is:

\begin{equation}
   \ln L = -n(\theta_0 + \theta_1 \times z) + \sum_i^{n} x_i \ln(\theta_0 + \theta_1 \times z) - \sum_i^{n} \ln(x_i!) 
\end{equation}

Gradient:

\begin{equation}
    \frac{\partial \log L}{\partial \theta_0} = -n + \frac{\sum_i^{n} x_i}{\theta_0 + \theta_1 \times z}   
\end{equation}

\begin{equation}
    \frac{\partial \log L}{\partial \theta_1} = -n z + \frac{\sum_i^{n} x_i z}{\theta_0 + \theta_1 \times z}   
\end{equation}

Hessian:

\begin{equation}
    H = \begin{bmatrix}
    \frac{\partial^2 \log L}{\partial \theta_0^2} & \frac{\partial^2 \log L}{\partial \theta_0 \partial \theta_1} \\
    \frac{\partial^2 \log L}{\partial \theta_0 \partial \theta_1} & \frac{\partial^2 \log L}{\partial \theta_1^2}
  \end{bmatrix}       
\end{equation}


\begin{equation}
    H = \begin{bmatrix}
    -\frac{\sum_i^{n} x_i}{(\theta_0 + \theta_1 \times z)^2} & -\frac{\sum_i^{n} x_i z}{(\theta_0 + \theta_1 \times z)^2} \\
    -\frac{\sum_i^{n} x_i z}{(\theta_0 + \theta_1 \times z)^2} & -\frac{\sum_i^{n} x_i z^2}{(\theta_0 + \theta_1 \times z)^2}
  \end{bmatrix}       
\end{equation}

 5. Obtain MLE of $\boldsymbol{\theta} = (\theta_0, \theta_1)$ using Newton-Raphson method (gradient and hessian should be applied).

In [43]:
%%R
## log-likelihood function
ll <- function(param, x, z) {
  m <- -10000*(param[1] + param[2]*z) + sum(x)*log(param[1] + param[2]*z)-log(factorial(x))
  m
}

## gradient
grad <- function(param, x, z)  {
  g <- numeric(2)
  g[1] <- -10000 + sum(x) / (param[1] + param[2]*z)
  g[2] <- -10000*z + sum(x)*z / (param[1] + param[2]*z)
  g
}


## hessian
hess <- function(param, x, z) {
  h <- matrix(0, nrow = 2, ncol = 2)
  h[1, 1] <- unique(-sum(x)/(param[1] + param[2]*z)^2)[1]
  h[1, 2] <- unique(-(sum(x)*z)/(param[1] + param[2]*z)^2)[1] 
  h[2, 1] <- h[1, 2]
  h[2, 2] <- unique(-(sum(x)*z^2)/(param[1] + param[2]*z)^2)[1]
  h
}

## with gradient and hessian
results <- maxLik(logLik = ll,  grad = grad, 
            hess = hess, start = c(1, 1), x = x, z = z, method = "NR")

summary(results)

--------------------------------------------
Maximum Likelihood estimation
Newton-Raphson maximisation, 6 iterations
Return code 1: gradient close to zero (gradtol)
Log-Likelihood: -103702437 
2  free parameters
Estimates:
     Estimate Std. error t value Pr(> t)
[1,]   0.4236        Inf       0       1
[2,]   0.4236        Inf       0       1
--------------------------------------------
